 **Screening Resumes based on Job Description**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk

In [ ]:
df_resume=pd.read_csv('../input/resume-dataset/UpdatedResumeDataSet.csv',encoding='utf-8')
df_resume.head(10)

In [ ]:
df_resume.shape

In [ ]:
print("Category:\n",df_resume['Category'].unique())
print("There are total ",len(df_resume['Category'].value_counts())," Categories.")

In [ ]:
category=df_resume['Category'].value_counts().reset_index()
category

plt.figure(figsize=(15,15))
plt.pie(category['Category'],labels=category['index'], colors=None, autopct='%1.1f%%')
plt.show()

In [ ]:
stopwords_english=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
def process_text(text):
    text1=re.sub('httpS+s*','',text)
    text2=re.sub(r'[^\w\s]','',text1)
    text3=nltk.word_tokenize(text2)
    modified_text=[word.lower() for word in text3]
    clean_text=[word for word in modified_text if(word not in stopwords_english and len(word)>2)]
    return clean_text

In [ ]:
#Adding new column
df_resume["Clean_resume"]=""

In [ ]:
df_resume['Clean_resume'] = df_resume['Resume'].apply(lambda x:process_text(x))
df_resume.head()

In [ ]:
df_resume['Clean_resume'] = df_resume['Clean_resume'].apply(lambda x:' '.join(x))
df_resume

In [ ]:
#Training and Testing data
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
requiredTarget=df_resume['Category'].values
requiredText=df_resume['Clean_resume'].values

Vectorizer=TfidfVectorizer(sublinear_tf=True, max_features=300, stop_words='english')
Vectorizer.fit(requiredText)
WordFeatures=Vectorizer.transform(requiredText)
X_train,X_test,y_train,y_test=train_test_split(WordFeatures,requiredTarget,test_size=0.2,
                                               random_state=10,shuffle=True)


In [ ]:
#Prediction Model
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
model=OneVsRestClassifier(KNeighborsClassifier())
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

In [ ]:
#Calculating Accuracy
classifier=OneVsRestClassifier(KNeighborsClassifier())
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)
print("Accuracy on Training set:{:.2f}".format(classifier.score(X_train,y_train)))
print("\nAccuracy on Test set:{:.2f}".format(classifier.score(X_test,y_test)))
                    

In [ ]:
print("\n Classification report based on Jobs:\n%s\n" % (metrics.classification_report(y_test, prediction)))